In [140]:
import pandas as pd
import os
import datetime
import random
from math import radians, degrees, sin, cos, asin, acos, sqrt

In [141]:
pd.set_option("display.max_columns", None)

In [143]:
data = {}
param = {}
input_directory = '../temp'
param_directory = '../param'
for _filename in os.listdir(input_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(input_directory+'/'+_filename,  header = None)
    filetype = _filename.split('-')[1]
    column_dict = { 
            'sked': ['from', 'to', 'al','fln', 'actype', 'depday','dep','arrday','arr'],
            'comp': ['from', 'to', 'al', 'fln','actype', 'depday','dep','arrday','arr'],
            'demand': ['orig', 'dest', 'unit','unit_vol', 'rev'],
            'demand_curve': ['orig', 'dest', 'ttt_1','ttt_2', 'ttt_3','ttt_4'],          
            'route_cost': ['from','to', 'actype', 'value'],
            'airport_cost': ['ap', 'actype', 'value'],
            'config': ['actype', 'unit_cap', 'vol_cap']
    }
    df_data.columns = column_dict[filetype]
    data[filetype] = df_data
    
for _filename in os.listdir(param_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(param_directory+'/'+_filename,  header = None)
    column_dict = { 
            'network' : ['stops_allowed'],
            'connections': ['hub', 'minct', 'maxct'],
            'preferences' : ['from', 'to', 'stop_penalty'],
            'airports': ["country_code","region_name","iata","icao","airport","latitude","longitude"]
    }
    df_data.columns = column_dict[_filename]
    param[_filename] = df_data

test-airport_cost
test-comp
test-config
test-demand
test-demand_curve
test-route_cost
test-sked
airports
connections
preferences


In [144]:
def gcd(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    return 6371 * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

In [151]:
#inputs
your_sked = data['sked']
comp_sked = data['comp']
data_config = data['config']

param_connect = param['connections']
param_pref = param['preferences']
param_pref['od'] = param_pref["from"]+param_pref["to"]
param_airport = param['airports']
param_airport['latitude']= param_airport['latitude'].astype(float)
param_airport['longitude']= param_airport['longitude'].astype(float)

max_stop = 2

your_sked = your_sked.merge(data_config, on='actype')
comp_sked = comp_sked.merge(data_config, on='actype')


full_sked = pd.concat([your_sked,comp_sked])
full_sked["dep"] = pd.to_datetime(full_sked["dep"]) + full_sked["depday"]*datetime.timedelta(days=1)
full_sked["arr"] = pd.to_datetime(full_sked["arr"]) + full_sked["arrday"]*datetime.timedelta(days=1)
full_sked["id"] = full_sked['al'] + full_sked['fln'].astype('str')+"/"+full_sked['depday'].astype('str')
full_sked = full_sked.merge(param_airport[['iata','latitude','longitude']].add_suffix('_from'),left_on=['from'], right_on=['iata_from'] )
full_sked = full_sked.merge(param_airport[['iata','latitude','longitude']].add_suffix('_to'),left_on=['to'], right_on=['iata_to'])

full_sked["unit_load"] = 0
full_sked["vol_load"] = 0
full_sked["netw_rev"] = 0
full_sked["pror_rev"]=0

full_sked["dist"] = full_sked.apply(lambda x: gcd(x['longitude_from'], x['latitude_from'],x['longitude_to'], x['latitude_to']), axis=1)

param_connect['minct'] = pd.to_timedelta(param_connect['minct'])
param_connect['maxct'] = pd.to_timedelta(param_connect['maxct'])

df_connect = full_sked.add_suffix('_1').reset_index(drop=True)
df_connect['nbstops'] = 0

list_itin = {}
list_itin[0] = df_connect.copy()
list_itin[0]['unit_demand']= 0
list_itin[0]['vol_demand']= 0
list_itin[0]['rev']= 0
list_itin[0]['tot_dist']=list_itin[0]['dist_1']

#build itineraries
for i in range (1,max_stop+1):
    cnx_p = param_connect.copy()
    itins = list_itin[i-1].merge(full_sked.add_suffix('_'+str(i+1)), how= "cross")
    
    
    itins['cnx_time_'+str(i)] = itins['dep_'+str(i+1)] - itins['arr_'+str(i)]
    
    cnx_p.columns = ['to_'+str(i), 'minct_'+str(i), 'maxct_'+str(i)]
    
    #filter connections
    itins.drop(itins[itins['to_'+str(i)] != itins['from_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(i)] == itins['to_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(1)] == itins['to_'+str(i+1)]].index, inplace=True)
    
    itins = itins.merge(cnx_p, on = ['to_'+str(i)])
    
    itins.drop(itins[itins['cnx_time_'+str(i)] < itins['minct_'+str(i)]].index, inplace=True)
    itins.drop(itins[itins['cnx_time_'+str(i)] > itins['maxct_'+str(i)]].index, inplace=True)
    
    
    itins['nbstops'] = i
    itins['tot_dist'] += itins['dist_'+str(i+1)]
    list_itin[i] = itins


#add attributes
for i in range(0,max_stop+1):
    list_itin[i]["travel_time"] = list_itin[i]["arr_"+str(i+1)]-list_itin[i]["dep_1"]
    list_itin[i]["od"] = list_itin[i]["from_1"]+list_itin[i]["to_"+str(i+1)]
    list_itin[i]["index"] = list_itin[i].index
    list_itin[i]['itin_id'] = list_itin[i]["nbstops"].astype(str)+"-"+list_itin[i]["index"].astype(str)
    list_itin[i] = list_itin[i].merge(param_pref, on="od")

    #calculate score
    list_itin[i]["score"] = list_itin[i]["travel_time"].dt.seconds+list_itin[i]["nbstops"]*list_itin[i]["stop_penalty"]*3600

    


In [152]:
def build_options(list_itin):
    od_itin = {}
    list_itin_summary = pd.DataFrame(columns=["itin_id","index", "od","travel_time", "nbstops","score"])

    for i in range(0,max_stop+1):
        list_itin_summary= pd.concat([list_itin_summary,list_itin[i][["itin_id","index", "od","travel_time", "nbstops","score"]]]) 
    #organize data by OD
    for od in param_pref.od.unique():    
        od_itin[od] = list_itin_summary[list_itin_summary['od']==od].copy()
    return list_itin_summary, od_itin

list_itin_summary, od_itin = build_options(list_itin)

In [153]:
#variables
timep = 4

#define user arrivals
demand_rand = {}
demand_curve = data['demand_curve']
demand = data['demand']
demand['od'] = demand['orig']+demand['dest']
demand_by_ttt = pd.merge(demand, demand_curve, on=["orig","dest"])

for i in range(1,1+timep):
    demand_by_ttt['d_ttt_'+str(i)]=(demand_by_ttt['ttt_'+str(i)]*demand_by_ttt['unit']).astype('int')

#shuffle pax

for i in range(1,1+timep):
    randm = []
    for indx, row in demand_by_ttt.iterrows():
        for k in range(1,row['d_ttt_'+str(i)]):
            randm.append(row['orig']+row['dest'])
    random.shuffle(randm)
    demand_rand[i] = randm
    

In [154]:
spill = dict.fromkeys(demand['od'].unique(),0)

avail_list_itin = {}
for key in list_itin:   
    avail_list_itin[key] = list_itin[key].copy()

avail_od_itin = {}
for key in od_itin:
    avail_od_itin[key] = od_itin[key].copy()

avail_list_itin_summary = list_itin_summary.copy()

###########################################################################################
#########################  TESTING WITH ONLY 1 TIMEPERIOD BUT NEED TO LOOP ################
###########################################################################################
list_pax = demand_rand[1]


#choose an itin randomly among available ones
for od_select in list_pax:
    if len(avail_od_itin[od_select])>0 :
        choice = random.choices(list(avail_od_itin[od_select]["itin_id"]), weights = list(avail_od_itin[od_select]["score"]), k=1)
        stop_index = choice[0].split('-')
        choice_stops = int(stop_index[0])
        choice_index = int(stop_index[1])  
        itin_selected = avail_list_itin[choice_stops]['index']==choice_index
        
        #add pax and revenue to itinerary 
        itin_selected_orig_list = (list_itin[choice_stops]['index']==choice_index)
        list_itin[choice_stops].loc[itin_selected_orig_list,'unit_demand'] += 1
        list_itin[choice_stops].loc[itin_selected_orig_list,'vol_demand'] += demand.loc[demand['od']==od_select,'unit_vol'].values[0]
        list_itin[choice_stops].loc[itin_selected_orig_list,'rev'] += demand.loc[demand['od']==od_select,'rev'].values[0]
        
        
        #add pax to flights
        for i in range(0, choice_stops+1):
            if len(avail_list_itin[choice_stops].loc[itin_selected,'id_'+str(i+1)])== 0:
                break
            flights_chosen = avail_list_itin[choice_stops].loc[itin_selected,'id_'+str(i+1)].values[0]
            full_sked.loc[full_sked['id']==flights_chosen,'unit_load' ] += 1
            full_sked.loc[full_sked['id']==flights_chosen,'vol_load' ] += demand.loc[demand['od']==od_select,'unit_vol'].values[0]
            full_sked.loc[full_sked['id']==flights_chosen,'netw_rev' ] += demand.loc[demand['od']==od_select,'rev'].values[0]
            full_sked.loc[full_sked['id']==flights_chosen,'pror_rev' ] += demand.loc[demand['od']==od_select,'rev'].values[0]*(
                list_itin[choice_stops].loc[itin_selected_orig_list,'dist_'+str(i+1)].values[0]/list_itin[choice_stops].loc[itin_selected_orig_list,'tot_dist'].values[0]
            )

        #remove closed flights
        for i in range(0, choice_stops+1):
            if (full_sked.loc[full_sked['id']==flights_chosen,'unit_load'].values[0] == full_sked.loc[full_sked['id']==flights_chosen,'unit_cap'].values[0] or
                full_sked.loc[full_sked['id']==flights_chosen,'vol_load'].values[0] >= full_sked.loc[full_sked['id']==flights_chosen,'vol_cap'].values[0]  ) :
                for j in range(0, max_stop+1):
                    for k in range(0,j+1):
                        avail_list_itin[j].drop(avail_list_itin[j][avail_list_itin[j]['id_'+str(k+1)] == flights_chosen].index, inplace=True)
                avail_list_itin_summary, avail_od_itin = build_options(avail_list_itin)
                
    else:
        #log spill
        spill[od_select] += 1 
        


In [155]:
full_sked

,from,to,al,fln,actype,depday,dep,arrday,arr,unit_cap,vol_cap,id,iata_from,latitude_from,longitude_from,iata_to,latitude_to,longitude_to,unit_load,vol_load,netw_rev,pror_rev,dist
0,DUS,TXL,FF,1,737,1,2022-02-08 10:52:00,1,2022-02-08 12:24:00,50,570,FF1/1,DUS,51.289501,6.766780,TXL,52.559700,13.287700,48,576,480,480.000000,468.744977
1,DUS,TXL,LH,1,737,1,2022-02-08 10:00:00,1,2022-02-08 12:00:00,50,570,LH1/1,DUS,51.289501,6.766780,TXL,52.559700,13.287700,48,576,480,480.000000,468.744977
2,CDG,TXL,AF,21,737,1,2022-02-08 18:14:00,1,2022-02-08 19:15:00,50,570,AF21/1,CDG,49.012798,2.550000,TXL,52.559700,13.287700,27,324,270,184.702179,850.497614
3,FCO,TXL,AZ,33,737,2,2022-02-09 09:26:00,2,2022-02-09 10:15:00,50,570,AZ33/2,FCO,41.800278,12.238889,TXL,52.559700,13.287700,0,0,0,0.000000,1198.981545
4,DUS,CDG,FF,2,737,1,2022-02-08 14:14:00,1,2022-02-08 17:00:00,50,570,FF2/1,DUS,51.289501,6.766780,CDG,49.012798,2.550000,32,582,554,458.228500,392.770642
5,DUS,CDG,LH,2,737,1,2022-02-08 14:00:00,1,2022-02-08 17:00:00,50,570,LH2/1,DUS,51.289501,6.766780,CDG,49.012798,2.550000,31,570,544,455.069321,392.770642
6,TXL,CDG,AF,15,380,1,2022-02-08 10:19:00,1,2022-02-08 12:41:00,50,1200,AF15/1,TXL,52.559700,13.287700,CDG,49.012798,2.550000,3,36,225,99.927370,850.497614
7,FRA,CDG,AF,16,737,1,2022-02-08 14:43:00,1,2022-02-08 17:14:00,50,570,AF16/1,FRA,50.033333,8.570556,CDG,49.012798,2.550000,0,0,0,0.000000,449.002847
8,MAD,CDG,AF,18,737,1,2022-02-08 09:35:00,1,2022-02-08 10:59:00,50,570,AF18/1,MAD,40.471926,-3.562640,CDG,49.012798,2.550000,0,0,0,0.000000,1064.512891
9,BCN,CDG,AF,19,380,3,2022-02-10 09:02:00,3,2022-02-10 09:47:00,50,1200,AF19/3,BCN,41.297100,2.078460,CDG,49.012798,2.550000,0,0,0,0.000000,858.738012


In [126]:
list_itin[0]

,from_1,to_1,al_1,fln_1,actype_1,depday_1,dep_1,arrday_1,arr_1,unit_cap_1,vol_cap_1,id_1,iata_from_1,latitude_from_1,longitude_from_1,iata_to_1,latitude_to_1,longitude_to_1,unit_load_1,vol_load_1,netw_rev_1,pror_rev_1,dist_1,nbstops,unit_demand,vol_demand,rev,tot_dist,travel_time,od,index,itin_id,from,to,stop_penalty,score
0,DUS,TXL,FF,1,737,1,2022-02-08 10:52:00,1,2022-02-08 12:24:00,50,570,FF1/1,DUS,51.289501,6.766780,TXL,52.559700,13.287700,0,0,0,0,468.744977,0,48,576,480,468.744977,0 days 01:32:00,DUSTXL,0,0-0,DUS,TXL,1,5520
1,DUS,TXL,LH,1,737,1,2022-02-08 10:00:00,1,2022-02-08 12:00:00,50,570,LH1/1,DUS,51.289501,6.766780,TXL,52.559700,13.287700,0,0,0,0,468.744977,0,48,576,480,468.744977,0 days 02:00:00,DUSTXL,1,0-1,DUS,TXL,1,7200
2,DUS,CDG,FF,2,737,1,2022-02-08 14:14:00,1,2022-02-08 17:00:00,50,570,FF2/1,DUS,51.289501,6.766780,CDG,49.012798,2.550000,0,0,0,0,392.770642,0,25,575,575,392.770642,0 days 02:46:00,DUSCDG,4,0-4,DUS,CDG,1,9960
3,DUS,CDG,LH,2,737,1,2022-02-08 14:00:00,1,2022-02-08 17:00:00,50,570,LH2/1,DUS,51.289501,6.766780,CDG,49.012798,2.550000,0,0,0,0,392.770642,0,25,575,575,392.770642,0 days 03:00:00,DUSCDG,5,0-5,DUS,CDG,1,10800
4,DUS,FRA,FF,3,737,1,2022-02-08 10:49:00,1,2022-02-08 15:37:00,50,570,FF3/1,DUS,51.289501,6.766780,FRA,50.033333,8.570556,0,0,0,0,188.868092,0,2,624,46,188.868092,0 days 04:48:00,DUSFRA,13,0-13,DUS,FRA,1,17280
5,DUS,FRA,LH,3,737,1,2022-02-08 10:20:00,1,2022-02-08 15:00:00,50,570,LH3/1,DUS,51.289501,6.766780,FRA,50.033333,8.570556,0,0,0,0,188.868092,0,2,624,46,188.868092,0 days 04:40:00,DUSFRA,14,0-14,DUS,FRA,1,16800
6,DUS,MAD,FF,5,737,3,2022-02-10 09:24:00,3,2022-02-10 10:25:00,50,570,FF5/3,DUS,51.289501,6.766780,MAD,40.471926,-3.562640,0,0,0,0,1441.299684,0,5,615,160,1441.299684,0 days 01:01:00,DUSMAD,17,0-17,DUS,MAD,1,3660
7,DUS,MAD,LH,5,737,3,2022-02-10 08:30:00,3,2022-02-10 09:40:00,50,570,LH5/3,DUS,51.289501,6.766780,MAD,40.471926,-3.562640,0,0,0,0,1441.299684,0,5,615,160,1441.299684,0 days 01:10:00,DUSMAD,18,0-18,DUS,MAD,1,4200
8,CDG,MAD,AF,24,737,1,2022-02-08 16:53:00,1,2022-02-08 17:34:00,50,570,AF24/1,CDG,49.012798,2.550000,MAD,40.471926,-3.562640,0,0,0,0,1064.512891,0,5,615,435,1064.512891,0 days 00:41:00,CDGMAD,19,0-19,CDG,MAD,1,2460
9,FCO,MAD,AZ,37,737,4,2022-02-11 05:35:00,4,2022-02-11 06:43:00,50,570,AZ37/4,FCO,41.800278,12.238889,MAD,40.471926,-3.562640,0,0,0,0,1329.630400,0,48,576,3840,1329.630400,0 days 01:08:00,FCOMAD,20,0-20,FCO,MAD,1,4080


In [ ]:
itin_selected

In [ ]:
itin_selected = avail_list_itin[choice_stops]['index']==choice_index

In [ ]:
itin_selected

In [ ]:
spill